In [23]:
import numpy as np

L=3
n=[2,4,4,1]

W1= np.random.randn(n[1],n[0])
W2= np.random.randn(n[2],n[1])
W3= np.random.randn(n[3],n[2])

b1=np.random.randn(n[1],1)
b2=np.random.randn(n[2],1)
b3=np.random.randn(n[3],1)

In [24]:
def my_input_data():
    
    X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
    [1, 0],
    [0, 1],
    [0, 0],
    [1, 1],
    [0, 1],
    [1, 0],
    [1, 1],
    [0, 0],
    [0, 1],
    [0, 0],
    [1, 0],
    [1, 1],
    [1, 0],
    [0, 1],
    [1, 1],
    [0, 0]
])
    
    Yo = np.array([0,1,1,0,1,1,0,0,1,1,0,0,1,0,1,0,1,1,0,0])
    m=20
    A0= X.T
    Y=Yo.reshape(n[L],m)
    
    return A0, Y

In [25]:
# since this is a binary classification problem the cost function is binary cross-entropy
def cost(y_hat,y):
    
    loss = - y*np.log(y_hat) - (1-y)*np.log(1-y_hat)
    
    sum_of_loss = np.sum(loss,axis=1)
    
    return sum_of_loss
    

In [26]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [27]:
def feed_forward(A0):
    z1= W1 @ A0 +b1
    A1 = sigmoid(z1)
    
    z2 = W2 @ A1 + b2
    A2 = sigmoid(z2)
    
    z3 = W3 @ A2 + b3
    A3 = sigmoid(z3)
    
    cache = {
        'A0': A0,
        'A1': A1,
        'A2': A2,
        'A3': A3,
    }
    
    return A3, cache

In [28]:
def backprop_l3(y_hat,y,m,a2,w3):
    
    a3 = y_hat
    dc_dz3 = 1/m * (a3-y)
    
    assert dc_dz3.shape == (n[3], m)
    
    dz3_dw3 = a2
    assert dz3_dw3.shape ==(n[2], m)
    
    dc_dw3 = dc_dz3 @ dz3_dw3.T
    
    assert dc_dw3.shape == (n[3], n[2])
    
    dc_db3 = np.sum(dc_dz3, axis=1, keepdims=True)
    assert dc_db3.shape == (n[3], 1)
    
    dz3_da2 = w3
    
    dc_da2 = w3.T @ dc_dz3
    
    assert dc_da2.shape == (n[2], m)
    
    return dc_dw3, dc_db3, dc_da2

def backprop_l2(dc_da2, a1,a2,w2, m):
    
    da2_dz2 = a2*(1-a2)
    
    dc_dz2 = dc_da2 * da2_dz2
    assert dc_dz2.shape == (n[2], m)
    
    dz2_dw2 = a1
    assert dz2_dw2.shape == (n[1], m)
    
    dc_dw2 = dc_dz2 @ dz2_dw2.T
    assert dc_dw2.shape == (n[2], n[1])
    
    dc_db2 = np.sum(dc_dz2, axis=1, keepdims=True)
    
    assert dc_db2.shape == (n[2], 1)
    
    dz2_da1 = w2
    dc_da1 = w2.T @ dc_dz2
    assert dc_da1.shape == (n[1], m)
    
    return dc_dw2, dc_db2, dc_da1

def backprop_l1(dc_da1, a0, a1, w1, m):
    da1_dz1 = a1 * (1 - a1)
    
    dc_dz1 = dc_da1 * da1_dz1
    assert dc_dz1.shape == (n[1], m)
    
    dz1_dw1 = a0
    assert dz1_dw1.shape == (n[0], m)
    
    dc_dw1 = dc_dz1 @ dz1_dw1.T
    assert dc_dw1.shape == (n[1], n[0])
    
    dc_db1 = np.sum(dc_dz1, axis=1, keepdims=True)
    assert dc_db1.shape == (n[1], 1)
    
    dz1_da0 = w1
    dc_da0 = w1.T @ dc_dz1
    assert dc_da0.shape == (n[0], m)
    
    return dc_dw1, dc_db1, dc_da0

In [37]:
def train_my_data():
    global W1, W2, W3, b1, b2, b3
    
    epochs = 5000
    learning_rate = 0.1
    costs = []
    
    for e in range(epochs):
        y_hat,cache = feed_forward(A0)
        
        error = cost(y_hat,Y)
        
        costs.append(error)
        
        dc_dw3, dc_db3, dc_da2 = backprop_l3(y_hat, Y, m, cache['A2'], W3)
        
        dc_dw2, dc_db2, dc_da1 = backprop_l2(dc_da2, cache['A1'], cache['A2'], W2, m)
        
        dc_dw1, dc_db1, dc_da0 = backprop_l1(dc_da1, cache['A0'], cache['A1'], W1, m)
        
        W3 -= learning_rate * dc_dw3
        W2 -= learning_rate * dc_dw2
        W1 -= learning_rate * dc_dw1
        
        b3 -= learning_rate * dc_db3
        b2 -= learning_rate * dc_db2
        b1 -= learning_rate * dc_db1
        
        if e%100 == 0:
            print(f'Epoch {e}, Cost: {np.mean(costs)}')
            
    return costs
    

In [38]:
A0, Y = my_input_data()  
m = A0.shape[1]    
costs = train_my_data()  

Epoch 0, Cost: 13.840172140838716
Epoch 100, Cost: 13.834829789129456
Epoch 200, Cost: 13.829117057036346
Epoch 300, Cost: 13.822943659737007
Epoch 400, Cost: 13.816204547809289
Epoch 500, Cost: 13.808775594589479
Epoch 600, Cost: 13.80050812281375
Epoch 700, Cost: 13.79122188342921
Epoch 800, Cost: 13.780695980627673
Epoch 900, Cost: 13.768657086386467
Epoch 1000, Cost: 13.754764105428576
Epoch 1100, Cost: 13.738588250293514
Epoch 1200, Cost: 13.719587309007476
Epoch 1300, Cost: 13.697072841326413
Epoch 1400, Cost: 13.670169340917013
Epoch 1500, Cost: 13.637765385482158
Epoch 1600, Cost: 13.598458698664146
Epoch 1700, Cost: 13.550499185281504
Epoch 1800, Cost: 13.49173334671741
Epoch 1900, Cost: 13.419545190277328
Epoch 2000, Cost: 13.330776551356323
Epoch 2100, Cost: 13.221627743412933
Epoch 2200, Cost: 13.08764064079104
Epoch 2300, Cost: 12.924034712273818
Epoch 2400, Cost: 12.726791704586182
Epoch 2500, Cost: 12.4946449664483
Epoch 2600, Cost: 12.230943025658577
Epoch 2700, Cost: 1

In [39]:
y_hat, _ = feed_forward(A0)

predicted_values = np.round(y_hat).astype(int)  
actual_values = np.round(Y).astype(int)       

print("Predictions:")
print(predicted_values.flatten())
print("Actual:")
print(actual_values.flatten())  

correct = (predicted_values == actual_values).sum()
total = predicted_values.size
accuracy = (correct / total) * 100
print(f"Accuracy: {accuracy:.2f}%")

Predictions:
[0 1 1 0 1 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0]
Actual:
[0 1 1 0 1 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0]
Accuracy: 100.00%
